In [236]:
import pandas as pd
import numpy as numpy
import json 
import os 
import pylab as pl
import urllib
import requests
import geopandas as gp
import fiona
import shapely
from fiona.crs import from_epsg
import os
import pandas as pd
import urllib
import requests
import json
import sys
import io
import shapely
import base64
import urllib.request
import functools
import geopandas as gp
import fiona
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## get adjusted day population and race population by precinct (from csv or acs)

In [237]:
myAPI = '90341a28f02794c77f6e3abb153a4952a0bb480b'

In [238]:
key11 = 'B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_006E,B02001_005E,B02001_007E,B02001_008E'

keyB = {'B02001_001E':'tot_w_race','B02001_002E':'white','B02001_003E': 'black', 'B02001_004E':'am_indian_alask_native',  
       'B02001_006E':'nat_hawai_pac_isl','B02001_005E':'asian','B02001_007E':'other_race', 'B02001_008E':'two_more_race'}

In [239]:
#B VARIABLES

year = '2013'

counties = [ 5, 47, 61, 81, 85]

url = "https://api.census.gov/data/2017/acs/acs5/?get=NAME," \
    + key11 + "&for=tract:*&in=state:36&in=county:*&key=" + myAPI
resp = requests.request('GET', url).content
acs_race= pd.read_csv(io.StringIO(resp.decode('utf-8').replace('[','').replace(']','')))

acs_race = acs_race[acs_race['county'].isin(counties)]

acs_race.rename(columns=keyB, inplace=True)
acs_race.head(1)

,NAME,tot_w_race,white,black,am_indian_alask_native,nat_hawai_pac_isl,asian,other_race,two_more_race,state,county,tract,Unnamed: 12
0,"Census Tract 119, Queens County, New York",1565,1104,10,16,0,128,205,102,36,81,11900,NaN


In [240]:
path= '/Users/katharinevoorhees/patrol_allocation/tract_precinct_key.csv'
prec_key = pd.read_csv(path)
prec_key['bcode'] = prec_key['bcode'].astype('str')
prec_key['cnty_num'] = prec_key['bcode'].str.slice(-3,)
prec_key['cnty_num'] = prec_key['cnty_num'].astype('int')

In [241]:
race_data_precinct = acs_race.merge(prec_key, how='left',left_on=['county','tract'], 
                                right_on=['cnty_num','tractnum'] )
race_data_precinct.shape

(2167, 28)

In [242]:
data = race_data_precinct.groupby('precinct').sum().reset_index()

In [243]:
data = data.loc[:,'precinct':'two_more_race']
data.head()

,precinct,tot_w_race,white,black,am_indian_alask_native,nat_hawai_pac_isl,asian,other_race,two_more_race
0,1.0,73428,55177,1909,80,10,12286,1075,2891
1,5.0,57974,16413,3866,303,5,33602,2020,1765
2,6.0,62780,53498,1334,84,4,5406,612,1842
3,7.0,48995,17621,4467,274,0,15084,9559,1990
4,9.0,74187,46361,6311,133,65,10158,8889,2270


In [244]:
#import adjusted daytime population for 2013 (previously calculated)

pop = pd.read_csv('precinct_population_dayadjust.csv')
pop.head(3)

,Unnamed: 0,precinct,resident_pop_x,county,day_allocation,adjusted_day_precinct
0,0,1.0,66751,61.0,271932.341967,338683.341967
1,1,5.0,57779,61.0,59919.357234,117698.357234
2,2,6.0,61230,61.0,40623.963767,101853.963767


In [245]:
data = data.merge(pop, on='precinct')

In [246]:
data.shape

(76, 14)

In [247]:
data.head()

,precinct,tot_w_race,white,black,am_indian_alask_native,nat_hawai_pac_isl,asian,other_race,two_more_race,Unnamed: 0,resident_pop_x,county,day_allocation,adjusted_day_precinct
0,1.0,73428,55177,1909,80,10,12286,1075,2891,0,66751,61.0,271932.341967,338683.341967
1,5.0,57974,16413,3866,303,5,33602,2020,1765,1,57779,61.0,59919.357234,117698.357234
2,6.0,62780,53498,1334,84,4,5406,612,1842,2,61230,61.0,40623.963767,101853.963767
3,7.0,48995,17621,4467,274,0,15084,9559,1990,3,51539,61.0,14779.598969,66318.598969
4,9.0,74187,46361,6311,133,65,10158,8889,2270,4,77426,61.0,26164.362216,103590.362216


### start model fairness test

In [248]:
#read in and merge model data

pop_MODEL = pd.read_csv('adjusted_day_MODEL_RESULTS.csv')
pop_MODEL = pop_MODEL[['precinct','MODEL_allocation']]
pop_MODEL.head(3)

,precinct,MODEL_allocation
0,1.0,653.0
1,5.0,227.0
2,6.0,196.0


In [249]:
data = data.merge(pop_MODEL, on='precinct')

In [250]:
data['other_agg'] = data['am_indian_alask_native'] + data['nat_hawai_pac_isl'] + data['other_race'] \
        + data['two_more_race']

In [251]:
data.head()

,precinct,tot_w_race,white,black,am_indian_alask_native,nat_hawai_pac_isl,asian,other_race,two_more_race,Unnamed: 0,resident_pop_x,county,day_allocation,adjusted_day_precinct,MODEL_allocation,other_agg
0,1.0,73428,55177,1909,80,10,12286,1075,2891,0,66751,61.0,271932.341967,338683.341967,653.0,4056
1,5.0,57974,16413,3866,303,5,33602,2020,1765,1,57779,61.0,59919.357234,117698.357234,227.0,4093
2,6.0,62780,53498,1334,84,4,5406,612,1842,2,61230,61.0,40623.963767,101853.963767,196.0,2542
3,7.0,48995,17621,4467,274,0,15084,9559,1990,3,51539,61.0,14779.598969,66318.598969,128.0,11823
4,9.0,74187,46361,6311,133,65,10158,8889,2270,4,77426,61.0,26164.362216,103590.362216,200.0,11357


In [252]:
# calculate potential model exposure (officers/people(adjusted_day) • # of group in precinct) vs proportion of total population for each race group

data['precinct_likelihood'] = (data['MODEL_allocation']/data['resident_pop_x'])

data['lik#_white'] = data['white'] * data['precinct_likelihood']
data['lik#_black'] = data['black'] * data['precinct_likelihood']
data['lik#_asian'] = data['asian'] * data['precinct_likelihood']
data['lik#_other'] = data['other_agg'] * data['precinct_likelihood']

lk_white = data['lik#_white'].sum()
lk_black = data['lik#_black'].sum()
lk_asian = data['lik#_asian'].sum()
lk_other = data['lik#_other'].sum()

total_likeli = likeli_white + likeli_black + likeli_asian + likeli_other

model_expos_white = lk_white/total_likeli
model_expos_black = lk_black/total_likeli 
model_expos_asian = lk_asian/total_likeli
model_expos_other = lk_other/total_likeli

percent_white = data['white'].sum()/data['tot_w_race'].sum()
percent_black = data['black'].sum()/data['tot_w_race'].sum()
percent_asian = data['asian'].sum()/data['tot_w_race'].sum()
percent_other = data['other_agg'].sum()/data['tot_w_race'].sum()

results = pd.DataFrame([['white',model_expos_white,percent_white],
                       ['black',model_expos_black,percent_black],
                       ['asian',model_expos_asian,percent_asian],
                       ['other',model_expos_other,percent_other]], columns=['race','model','population'])

results

,race,model,population
0,white,0.590180,0.423853
1,black,0.285156,0.246618
2,asian,0.187710,0.139221
3,other,0.225899,0.190307


#### this makes sense - the model was decided purely on population so population ratios should hold up city wide

In [253]:
#possible function for evaluating fairness - sum sq errors

def fairness_eval (df,colm,colp):
    sum_sq_difs = 0
    for i in df.iterrows():
        dif = (df[colm]-df[colp])**2
        sum_sq_difs += dif
    
    return np.round(sum_sq_difs.sum(),5)

In [254]:
fairness_eval(results,'model','population')

0.13107

In [255]:
###test the area model

area_model = pd.read_csv('area_model_results.csv')
area_model.head(3)

,Unnamed: 0,precinct,sq-km-calc
0,0,1.0,112.318729
1,1,5.0,42.960712
2,2,6.0,52.510118


In [256]:
data.drop(columns=['lik#_white',
       'lik#_black', 'lik#_asian', 'lik#_other','MODEL_allocation', 'precinct_likelihood'],
         inplace=True)

In [257]:
data = data.merge(area_model, on='precinct')
data.head()

,precinct,tot_w_race,white,black,am_indian_alask_native,nat_hawai_pac_isl,asian,other_race,two_more_race,Unnamed: 0_x,resident_pop_x,county,day_allocation,adjusted_day_precinct,other_agg,Unnamed: 0_y,sq-km-calc
0,1.0,73428,55177,1909,80,10,12286,1075,2891,0,66751,61.0,271932.341967,338683.341967,4056,0,112.318729
1,5.0,57974,16413,3866,303,5,33602,2020,1765,1,57779,61.0,59919.357234,117698.357234,4093,1,42.960712
2,6.0,62780,53498,1334,84,4,5406,612,1842,2,61230,61.0,40623.963767,101853.963767,2542,2,52.510118
3,7.0,48995,17621,4467,274,0,15084,9559,1990,3,51539,61.0,14779.598969,66318.598969,11823,5,43.612396
4,9.0,74187,46361,6311,133,65,10158,8889,2270,4,77426,61.0,26164.362216,103590.362216,11357,6,50.825765


In [258]:
#only the first line changes as long as the relevant df columns are the same

data['precinct_likelihood'] = (data['sq-km-calc']/data['resident_pop_x'])


data['lik#_white'] = data['white'] * data['precinct_likelihood']
data['lik#_black'] = data['black'] * data['precinct_likelihood']
data['lik#_asian'] = data['asian'] * data['precinct_likelihood']
data['lik#_other'] = data['other_agg'] * data['precinct_likelihood']

lk_white = data['lik#_white'].sum()
lk_black = data['lik#_black'].sum()
lk_asian = data['lik#_asian'].sum()
lk_other = data['lik#_other'].sum()

total_likeli = likeli_white + likeli_black + likeli_asian + likeli_other

model_expos_white = lk_white/total_likeli
model_expos_black = lk_black/total_likeli 
model_expos_asian = lk_asian/total_likeli
model_expos_other = lk_other/total_likeli

percent_white = data['white'].sum()/data['tot_w_race'].sum()
percent_black = data['black'].sum()/data['tot_w_race'].sum()
percent_asian = data['asian'].sum()/data['tot_w_race'].sum()
percent_other = data['other_agg'].sum()/data['tot_w_race'].sum()

results = pd.DataFrame([['white',model_expos_white,percent_white],
                       ['black',model_expos_black,percent_black],
                       ['asian',model_expos_asian,percent_asian],
                       ['other',model_expos_other,percent_other]], columns=['race','model','population'])

results2

,race,model,population
0,white,0.454424,0.423853
1,black,0.250978,0.246618
2,asian,0.139718,0.139221
3,other,0.154880,0.190307


In [259]:
fairness_eval(results2,'model','population')

0.00884